### Install required dependencies

In [1]:
%%capture
!pip install tqdm
!pip install seaborn

### Import dependecies

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
import traceback
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

### Initialize File Paths

In [3]:
time_fix_path = '/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/input_data/time_fixes.csv'
input_file_path = '/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/output_data/data_draft.csv'
output_file_path = '/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/output_data/data_draft_v2.csv'

In [4]:
df = pd.read_csv(input_file_path)
df.head(3)

,SubID,Burst,Weekend,Day,RespID,RespType,UserID,SurvName,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,MR_wake,MR_rise,MR_bed,MR_FlSlp,MR_SlpQlt,MR_DrnkNum,MR_DrnkDur,MR_rlx,MR_hngvr,MR_Embrssd,MR_agrsv,MR_vom,MR_hrt,MR_rmbr,MR_work,MR_rude,MR_soc,MR_exprs,MR_guilt,MR_expctd,MR_mood,MR_mrng,MR_prmsd,MR_more,MR_fght,MR_enrgtc,MR_PsdOut,MR_MsdClss,MR_Impls,MR_sleep,MR_weed,MR_plan,chr,anoy,upbt,sad,irit,hap,bor,cnt,RP_enrgz,RP_excit,RP_sedat,RP_slotht,RP_slug,RP_up,RP_crntloc,RP_othrloc,RP_wthothr,RP_NumOth,RP_OthTyp,RP_OthWho,RP_EnjInt,RP_ActCld,RP_ActAcc,RP_OthCld,RP_OthAcc,RP_Vrtl,RP_AlnLke,RP_AlnGd,RP_WshOth,RP_lnly,RP_rjct,RP_incl,RP_acpt,RP_vrtint,RP_vrtoth,RP_vrtwho,RP_vrtenj,RP_vrtcld,RP_VrtAcc,RP_VrtOthCld,RP_VrtOthAcc,RP_NotInt,RP_WshInt,RP_lonely,RP_reject,RP_included,RP_accepted,RP_said,RP_spent,RP_impt,RP_spur,RP_Drnk
0,2001,1,1,Thu,5ab425aa2c9b2f5baaa69eaf,Submission,5a8381835893ca7070656612,Morning Reports,Assessment,2018-03-22,17:49:33,2018-03-22,17:52:41,2018-03-22,05:00:00,07:30:00,07:40:00,12:00:00,1.0,4.0,1.0,CONDITION_SKIPPED,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,71.0,27.0,79.0,21.0,9.0,69.0,11.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,1,1,Fri,5ab528815831424890f49c8c,Submission,5a8381835893ca7070656612,Daytime Surveys,Assessment,2018-03-23,12:15:03,2018-03-23,12:17:04,2018-03-23,12:14:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,14.0,76.0,15.0,15.0,82.0,33.0,78.0,78.0,55.0,14.0,14.0,27.0,84.0,6.0,CONDITION_SKIPPED,1.0,11,"4,5,6,7",CONDITION_SKIPPED,59,41,42,58,35,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,33.0,67.0,2.0
2,2001,1,1,Fri,5ab53c5fe15db24566df3a98,Submission,5a8381835893ca7070656612,Daytime Surveys,Assessment,2018-03-23,13:40:33,2018-03-23,13:41:51,2018-03-23,13:39:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,13.0,77.0,13.0,29.0,83.0,17.0,76.0,82.0,35.0,27.0,13.0,24.0,75.0,6.0,CONDITION_SKIPPED,1.0,2,5,CONDITION_SKIPPED,71,82,19,69,82,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,0.0,0.0,2.0


In [5]:
# df.sample(50)

In [6]:
df[~df['MR_DrnkDur'].isna()].MR_DrnkDur.sample(10)

25955     CONDITION_SKIPPED
18419               2 hours
20565               6 hours
3337      CONDITION_SKIPPED
22706                45 min
8898               10 hours
7893               01:00:00
12495    2 hours 30 minutes
14052     CONDITION_SKIPPED
5893      CONDITION_SKIPPED
Name: MR_DrnkDur, dtype: object

### TO DO
- Fill in Daytime Survey values
- MR_DrnkDur Column
- MR_wake

### 1. Filling Daytime Survey Values

In [7]:
# Create a progress bar using tqdm
progress_bar = tqdm(total=len(df[df['SurvName'] == 'Daytime Surveys']))

# Iterate over rows with SurvName equal to 'Daytime Surveys'
for index, row in df[df['SurvName'] == 'Daytime Surveys'].iterrows():
    date = row['SubDate']
    
    morning_df = df[(df['SubDate'] == date) & (df['SurvName'] == 'Morning Reports')]
    if len(morning_df)==0:
        continue

    # Assuming that in a given day only one morning survey is available. Is this a sound assumption?
    morning_reports_row = morning_df.iloc[0]

    # Fill NaN values in the 'Daytime Surveys' row using corresponding 'Morning Reports' row
    for column in df.columns:
        if pd.isna(row[column]):
            df.loc[index, column] = morning_reports_row[column]

    progress_bar.update(1)

progress_bar.close()

100%|███████████████████████████████████▉| 22554/22593 [01:20<00:00, 280.96it/s]


### Normalizing MR_DrnkDur

In [8]:
df['MR_DrnkDur_original'] = df['MR_DrnkDur']

fixer_df = pd.read_csv(time_fix_path)
fixer_df = fixer_df.rename(columns={'From':'MR_DrnkDur', 'To':'MR_DrnkDur+'})

In [9]:
df = df.merge(fixer_df, on = 'MR_DrnkDur', how = 'left')
df['MR_DrnkDur'] = df['MR_DrnkDur+'].fillna(df['MR_DrnkDur'])
df = df.drop(columns = ['MR_DrnkDur+'])

In [10]:
def get_hours_minutes(duration_str):
    duration_str = str(duration_str)
    # Use regular expressions to extract hours and minutes
    hours = None
    minutes = None

    # Extract hours
    # hours_match = re.search(r'(\d+)\s*h.*?|(\d+)\s*hrs?|(^\d+)\s+\d+.*', duration_str)
    hours_match = re.search(r'(\d+\.?\d*)\s*h.*|(\d+):.*|(\d+\.?\d*)\s+\d+', duration_str)
    if hours_match:
        for group in hours_match.groups():
            if group is not None:
                hours = float(group)
                break

    # Extract minutes
    minutes_match = re.search(r'(\d+)\s*minutes+|(\d+)\s*min+|[\s:]+(\d+$)|.*:(\d+):.*', duration_str)
    if minutes_match:
        for group in minutes_match.groups():
            if group is not None:
                minutes = int(group)
                break
    
    if hours is None and minutes is None:
        num_match = re.search(r'(\d+\.?\d*)', duration_str)
        if num_match:
            for group in num_match.groups():
                if group is not None:
                    temp = float(group)
                    if temp < 10:
                        hours = temp
                    elif temp % 5 == 0 or temp >= 20:
                        minutes = temp
                    else:
                        hours = temp
                    break
    if hours is not None:
        if minutes is None:
            minutes = 0
        
        minutes += hours * 60
        hours = int(minutes // 60)
        minutes = int(minutes % 60)
    elif minutes is not None:
        hours = 0
        hours = int(minutes // 60)
        minutes = int(minutes % 60)
        
    
    return hours, minutes

def normalize_duration(duration_str):
    result = ''
    if not duration_str or duration_str=='CONDITION_SKIPPED' or duration_str=='nan' or pd.isna(duration_str):
        result = 'CONDITION_SKIPPED'
    
    else:
        hours, minutes = get_hours_minutes(duration_str)
        if hours is not None:
            result = f'{hours} hours'
        if minutes is not None:
            result += f' {minutes} minutes'

    return f'{result}'

def get_minutes(duration_str):
    result = ''
    
    if not duration_str or duration_str=='CONDITION_SKIPPED' or duration_str=='nan' or pd.isna(duration_str):
        return 'CONDITION_SKIPPED'
    else:
        hours, minutes = get_hours_minutes(duration_str)
        res = 0
        if hours is not None:
            res += hours * 60
        if minutes is not None:
            res += minutes
        if res == 0:
            return None
        return res

In [11]:
number_words_mapping = {
    "zero": "0",
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9",
    "ten": "10",
    "eleven": "11",
    "twelve": "12",
    "thirteen": "13",
    "fourteen": "14",
    "fifteen": "15",
    "sixteen": "16",
    "seventeen": "17",
    "eighteen": "18",
    "nineteen": "19",
    "twenty": "20",
    "thirty": "30",
    "hour and a half" : "1 hour 30 minutes"
}

def replace_number_words(df, column_name):
    for word, number in number_words_mapping.items():
        # Using regex to match whole words (\b denotes word boundary)
        regex_pattern = r'\b' + word + r'\b'
        df[column_name] = df[column_name].str.replace(regex_pattern, number, regex=True)
    return df

In [12]:
# Apply the normalization function to the DataFrame
df = replace_number_words(df, 'MR_DrnkDur')
df['MR_DrnkDur'] = df['MR_DrnkDur'].apply(normalize_duration)
df['MR_DrnkDur_min'] = df['MR_DrnkDur'].apply(get_minutes)

In [13]:
df[['MR_DrnkDur_original', 'MR_DrnkDur']].value_counts().reset_index().sample(30)

,MR_DrnkDur_original,MR_DrnkDur,count
219,02:00:00,2 hours 0 minutes,2
18,4 hours 0 minutes,4 hours 0 minutes,224
185,3 hours and 30 minutes,3 hours 30 minutes,2
329,"7 ,2",7 hours 0 minutes,1
121,14 hours,14 hours 0 minutes,5
278,06 hrs 00 mins,6 hours 0 minutes,1
475,35 minutes,0 hours 35 minutes,1
23,3hrs,3 hours 0 minutes,111
448,1 hour 30 mintes,1 hours 30 minutes,1
395,2 hr,2 hours 0 minutes,1


### Plot MR Time Columns

In [14]:

# time_columns = ['MR_wake', 'MR_rise', 'MR_bed']
# plt.figure(figsize=(8, 6))
# hour_labels = [f"{i:02d}:00:00" for i in range(25)]

# # Create a Seaborn color palette for multiple columns
# palette = sns.color_palette("hls", len(time_columns))

# for index, time_column in enumerate(time_columns):
#     # Plot the frequencies for each time column
#     temp_df = df[time_column].value_counts().reset_index().sort_values(by=[time_column]).reset_index(drop=True)
#     sns.barplot(x=temp_df[time_column], y=temp_df['count'], label=f'{time_column} vs. Frequency', color=palette[index])

# plt.xlabel('Time')
# plt.ylabel('Frequency')
# plt.title('Time vs. Frequency for MR time columns')

# # Show a legend to distinguish each line
# plt.legend()

# # Set x-tick positions and labels
# x_ticks = np.arange(0, len(hour_labels))
# plt.xticks(x_ticks, hour_labels, rotation=45)

# # Show the plot
# plt.tight_layout()
# plt.show()


In [15]:
df.to_csv(output_file_path,index=False)